In [3]:
# Load libraries
import networkx as nx
from networkx import *
import pandas as pd
import numpy as np
import math
import psycopg2
import sys
from pandas.io import sql
import sqlalchemy
import os
from sqlalchemy import create_engine, text
from sqlalchemy.types import INTEGER
from datetime import datetime
from configobj import ConfigObj
from tqdm.notebook import tqdm
tqdm.pandas()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.path_finder import solve_mw_path
from functions.utils import database

In [4]:
#Database connection information
config_path = "/home/jovyan/shared/rural_planner_refactoring/config_files/config_pe"

parser = ConfigObj(config_path)

sql_path = parser['sql_path']
country_folder = parser['country_folder']

owners = dict(parser['path_finder_fiber']['owners'])

schema = parser['path_finder_mw']['schema']
table_towers = parser['transport_by_tower_params']['table_infrastructure']
table_clusters = parser['clustering_params']['output_table']
output_table = parser['path_finder_mw']['output_table']
final_table = parser['path_finder_mw']['final_table']
table_line_of_sight = parser['path_finder_mw']['table_line_of_sight']

In [8]:
#Load edges
query_path = sql_path + '/' + country_folder + '/' + 'path_finder_mw_load_edges.sql'

with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_towers = table_towers, table_line_of_sight = table_line_of_sight)
    df_edges = pd.read_sql_query(query_formatted, con = db)

In [9]:
#Create graph
G = nx.Graph()

for z in range(0, len(df_edges)):
    G.add_edge(str(int(df_edges['tower_id_1'].iloc[z])),
               str(int(df_edges['tower_id_2'].iloc[z])))
    G.add_edge(str(int(df_edges['tower_id_2'].iloc[z])),
               str(int(df_edges['tower_id_1'].iloc[z])))
    
graphs = list(nx.connected_component_subgraphs(G))

In [11]:
#Load all fiber nodes and terminal nodes
query_path_fiber = sql_path + '/' + country_folder + '/' + 'path_finder_mw_load_fiber.sql'
query_path_terminal = sql_path + '/' + country_folder + '/' + 'path_finder_mw_load_terminal_nodes.sql'

with open(query_path_fiber) as file_fiber, open(query_path_terminal) as file_terminal, database(parser) as db:
    query = file_fiber.read()
    query_formatted = query.format(schema = schema,
                                   table_towers = table_towers)
    df_edges_fiber = pd.read_sql_query(query_formatted, con = db)
    
    query = file_terminal.read()
    query_formatted = query.format(schema = schema,
                                   table_towers = table_towers)
    df_edges_terminal = pd.read_sql_query(query_formatted, con = db)

In [14]:
#Calculate mw paths (takes less than 6h)
print(str(datetime.now()))
all_terminal_nodes = df_edges_terminal['tower_id'].astype(int).astype(str).tolist()

df_output = pd.DataFrame()

#One iteration for every connected independent subgraph
for k in range(0, len(graphs)):
    
    #Take one of the connected subgraphs and turn it into a data frame
    G1 = graphs[k]

    nodes_subgraph = list(G1.nodes())
    
        
    #This is the data frame with the edges associated to the fiber
    df_edges_fiber_all = df_edges_fiber.ix[df_edges_fiber['tower_id'].isin(nodes_subgraph),]    
    
    # Filter terminal nodes from current subgraph 
    terminal_nodes = { node for node in all_terminal_nodes if node in nodes_subgraph}
    terminal_nodes = list(terminal_nodes)

    if len(terminal_nodes)>0:
        #Calculate optimal path for every terminal node with Dijkstra's algorithm
        df_output_subgraph = pd.DataFrame({'node_id': terminal_nodes})

        for owner in owners.keys():    
            df_edges_fiber_owner =  df_edges_fiber_all.ix[df_edges_fiber_all['source'].isin(owners[owner]['sources'])]
            fiber_nodes_owner = df_edges_fiber_owner['tower_id'].astype(int).astype(str).tolist()

            # calculate optimal fiber path per owner
            df_output_subgraph = df_output_subgraph.join(df_output_subgraph.progress_apply(solve_mw_path, axis=1, args=(owner, fiber_nodes_owner, G1, df_edges)))
            print(str(owner)+ ' processed.')   

        df_output = df_output.append(df_output_subgraph)     
        del df_output_subgraph

2020-02-03 15:41:46.025181


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



movistar processed.



azteca processed.


...


third_party processed.



movistar processed.



azteca processed.



regional processed.



third_party processed.


In [17]:
output_cols = list(['node_id'])

for owner in owners.keys():
    owners_cols = list(['length_' + owner,
                   'path_' + owner, 
                   'fiber_node_' + owner])
    output_cols.extend(owners_cols)
    
sort_cols = ['node_id']
sort_cols.extend(['fiber_node_'+owner for owner in owners.keys()])

df_output = df_output[output_cols].sort_values(sort_cols).drop_duplicates('node_id').reset_index(drop = True)

In [ ]:
##export, replace arrays limits, cast variables to int/float, create geoms

query_path_export = sql_path + '/' + country_folder + '/' + 'path_finder_mw_export.sql'
query_path_final = sql_path + '/' + country_folder + '/' + 'path_finder_mw_final_table.sql'

with open(query_path_export) as file_export, open(query_path_final) as file_final, database(parser) as db:
    df_output.to_sql(output_table, con=db, if_exists = 'replace', schema = schema, index = False)
    query = file_export.read()
    query_formatted = query.format(schema = schema,
                                   output_table = output_table,
                                   table_towers = table_towers)
    db.execute(query_formatted)
    # Create final output
    query = file_final.read()
    query_formatted = query.format(schema = schema,
                                   final_table = final_table,
                                   table_clusters = table_clusters,
                                   table_towers = table_towers,
                                   output_table = output_table)
    db.execute(query_formatted)